In [109]:
def sum_of_squares(list1):
    return sum([element ** 2 for element in list1])

In [110]:
print sum_of_squares([1,2,3,4,5])
print sum_of_squares([])

55
0


# Poker Game Program

Each card has a rank and suite. For 5J, rank is 5, suite is J. <br>

Write a program that picks the best ranked hand.  <br>
The following is the list of different kinds of hand in ascending order of importance/rank

<b>0) Nothing or High Card:</b> <br>
When two hands fall in this category, we break the tie based on the highest card of each hand<br><br>


<b>1) pair:</b> <br>
Two of a kind. Others are all of different kind.<br><br>

<b>2)Two pair:</b><br>
Two of a kind x. Two more of same kind y<br><br>


<b>3)3 of a kind:</b><br>
 three cards have same rank<br><br>

<b>4) straight:</b><br>
The ranks of the hand form a sequence.<br><br>

<b>5) Flush:</b><br>
All cards in the hand have same suite.<br><br>

<b>6) Full House:</b><br>
Three cards of one rank. Two cards have another rank.<br><br>

<b>7) Four of a kind:</b><br>
four cards have same rank<br><br>

<b>8) Straigh flush:</b><br>
Straight and Flush.<br><br>

In [111]:
def card_ranks(hand):
    "Returns ranks of hand in reverse order"
    special_cards = {"T": 10, "J": 11, "Q": 12, "K" : 13, "A" : 14}
    return sorted([ int(card[0]) if card[0] not in special_cards else special_cards[card[0]]   for card in hand ],reverse = True)

def straight(ranks):
    "Returns whether the given ranks form a straight"    
    for i in range(0,len(ranks)-1):
        if ranks[i] != ranks[i+1] + 1:
            return False
    return True

def flush(hand):
    "Returns True if all cards have same suite"
    return len(set([card[1] for card in hand])) == 1

from collections import Counter            
def kind(n, ranks):
    "Return the rank that repeats n times. When there is no such unique rank, this function return False"
    counter = Counter(ranks)
    ranks_with_freq_n = [k for k,v in counter.iteritems() if v == n ] 
    if len(ranks_with_freq_n) == 1:
        return ranks_with_freq_n[0]
    else:
        return None
        

def two_pair(ranks):
    "Return reverse sorted two_pair for given rank. Else return None"
    counter = Counter(ranks)
    ranks_with_freq_n = [k for k,v in counter.iteritems() if v == 2 ] 
    if len(ranks_with_freq_n) == 2:
        return sorted(ranks_with_freq_n, reverse= True)
    else:
        return None
    
    
def hand_rank(hand):
    "Return the rank of given hand: hand_rank(hand) => value"
    ranks = card_ranks(hand)
    # card_ranks returns the ranks in reverse sorted order.
    if straight(ranks) and flush(hand):
        return (8,max(ranks))
    elif kind(4, ranks):
        return (7, kind(4,ranks),kind(1,ranks))
    elif kind(3, ranks) and kind(2, ranks):
        return (6, kind(3, ranks), kind(2, ranks))
    elif flush(hand):
        return (5, max(ranks))
    elif straight(ranks):
        return (4, max(ranks))
    elif kind(3, ranks):
        # TODO should include the two other numbers in the tuple
        return (3, kind(3, ranks), ranks)
    elif two_pair(ranks):
        return(2, two_pair(ranks),ranks)
    elif kind(2,ranks):
        return (1, kind(2, ranks), ranks)
    else:
        return (0, ranks)

def poker(hands):
    "Return the best hand: poker([hand,...]) => hand"
    return max(hands, key = hand_rank)

In [112]:
def test(poker):
    "Test cases for the functions in the poker program"
    #### TEST 1
    # straight flush
    sf = "6C 7C 8C 9C TC".split()
    # four ok a kinf
    fk = "9D 9H 9S 9C 7D".split()
    # full house
    fh = "TD TC TH 7C 7D".split()
    assert poker([sf,fk,fh]) == sf
    #### TEST 2
    assert poker([fk,fh]) == fk
    #### TEST 3
    assert poker([fh,fh]) == fh
    #### TEST 4
    assert poker([fh]) == fh
    #### TEST 5
    assert poker([sf] + 99*[fh]) == sf
    #### TEST CASES for hand_rank
    assert hand_rank(sf) == (8,10)
    assert hand_rank(fk) == (7,9,7)
    assert hand_rank(fh) == (6,10,7)
    return "tests pass"

In [113]:
print test(poker)

tests pass


# Further Modifications
# Modification1
Now we need to modify the Poker Program to accomodate an additional requirement.<br>
Till now we have been blindly mapping A to rank 14. But there are cases where the card player can use A as rank 1.<br><br>
<b>Example1:</b> "AS 2S 3S 4S 5C".split()   # A-5 straight <br>
A is used here with rank 1. the highest rank in this hand is 5.<br><br>
<b>Example2:</b> "AS 2S 3S 4S 6C".split()   # A high <br>
A is used here with rank 14.<br>




A will always map to 14, Except in the case of sequence 2,3,4,5,6 where A will map to 1.<br>
# Solution 1
Let's just handle that case explicitly in hand_rank<br><br>
Demerits of this solution are: <br>
                          1) Not Extensible:<br> 
                             a) If poker game changes to add a new type of hand,<br>
                                which is a variant of straight, then the explicit checking increasing.<br>
                             b) If straight hand is removed from the game, then we have to clean up the explicit<br>
                                check also.<br>
                             c) Code duplication of explicit handling<br><br>
Merits of this solution:<br>
                         1) Code change is proportional to Change in requirement<br>

# Solution 2
Another alternative is to change the card_ranks implementation<br>
 It is similar to above solution in all aspects. But the code is DRY(doesn't repeat itself).


In [114]:
##SOLUTION 1

def hand_rank1(hand):
    "Return the rank of given hand: hand_rank(hand) => value"
    # assume that hand has 5 cards. 
    #If it doesn't raise an exception
    if len(hand) != 5:
        raise 
    ranks = card_ranks(hand)
    # card_ranks returns the ranks in reverse sorted order.
    if straight(ranks) and flush(hand):
        return (8,max(ranks))
    # explicitly handle the sequence A,2,3,4,5
    elif ranks == [14,5,4,3,2] and flush(hand):
        return (8,5)
    elif kind(4, ranks):
        return (7, kind(4,ranks),kind(1,ranks))
    elif kind(3, ranks) and kind(2, ranks):
        return (6, kind(3, ranks), kind(2, ranks))
    elif flush(hand):
        return (5, max(ranks))
    elif straight(ranks):
        return (4, max(ranks))
    # explicitly handle the sequence A,2,3,4,5
    elif ranks == [14,5,4,3,2]:
        return (4,5)
    elif kind(3, ranks):
        return (3, kind(3, ranks), ranks)
    elif two_pair(ranks):
        return(2, two_pair(ranks),ranks)
    elif kind(2,ranks):
        return (1, kind(2, ranks), ranks)
    else:
        return (0, ranks)
    
def poker1(hands):
    "Return the best hand: poker([hand,...]) => hand"
    return max(hands, key = hand_rank1)

def more_tests(poker):
    sf_withA = "AS 2S 3S 4S 5S".split()  #A-5 straight flush
    # straight flush
    sf = "6C 7C 8C 9C TC".split()
    # four ok a kinf
    fk = "9D 9H 9S 9C 7D".split()
    # full house
    fh = "TD TC TH 7C 7D".split()
    assert poker([fk,sf_withA]) == sf_withA
    return "additional tests passed" 
print test(poker1)
print more_tests(poker1)    

tests pass
additional tests passed


In [117]:
## SOLUTION 2
def card_ranks(hand):
    "Returns ranks of hand in reverse order. Also assign A rank 1 to allow A-5 sequence"
    special_cards = {"T": 10, "J": 11, "Q": 12, "K" : 13, "A" : 14}
    ranks = sorted([ int(card[0]) if card[0] not in special_cards else special_cards[card[0]]   for card in hand ],reverse = True)
    return ranks if ranks != [14,5,4,3,2] else [5,4,3,2,1]

print test(poker)
print more_tests(poker)    

tests pass
additional tests passed


# Modification 2
How do we handle ties in the poker game?
modify poker function


In [147]:
def poker(hands):
    # Returns list of best hands: poker([hand,...]) => [hand,...]
    return allmax(hands,key=hand_rank)

def allmax(iterable,key=None):
    max_value = max(iterable) if key == None else max(iterable, key=key)
    # frequency * value
    return iterable.count(max_value) * [max_value]

In [148]:
print allmax([1,2,3,-4])
print allmax([1,2,3,-4],key=abs)
print allmax([1,2,3,-4,-4,-4],key=abs)


[3]
[-4]
[-4, -4, -4]
